In [1]:
import pandas as pd
helper = pd.read_csv(r'C:\Users\c3205\Documents\Code\Jupyter Notebooks\draw\draws\V6\helper.csv')

In [2]:
helper = helper[helper['DATE'] != '27-April-2025']

In [ ]:
helper['TEAM 1'] = helper['TEAM 1'].astype(str)
helper['TEAM 2'] = helper['TEAM 2'].astype(str)

helper['TEAM 1'] = helper['TEAM 1'].str.strip()
helper['TEAM 2'] = helper['TEAM 2'].str.strip()
helper['GRADE'] = helper['GRADE'].str.strip()

helper['TEAM 1'] = helper['TEAM 1'].str.replace('University Gentleman', 'University Gentlemen')
helper['TEAM 2'] = helper['TEAM 2'].str.replace('University Gentleman', 'University Gentlemen')

helper['TEAM 1'] = helper['TEAM 1'].str.replace('Unversity Seapigs', 'University Seapigs')
helper['TEAM 2'] = helper['TEAM 2'].str.replace('Unversity Seapigs', 'University Seapigs')

helper['TEAM 1'] = helper['TEAM 1'].str.replace('Unversity', 'University')
helper['TEAM 2'] = helper['TEAM 2'].str.replace('Unversity', 'University')

helper['TEAM 1'] = helper['TEAM 1'].str.replace('Universiity', 'University')
helper['TEAM 2'] = helper['TEAM 2'].str.replace('Universiity', 'University')

helper['TEAM 1'] = helper['TEAM 1'].str.replace('West Red', 'Wests Red')
helper['TEAM 2'] = helper['TEAM 2'].str.replace('West Red', 'Wests Red')

helper['TEAM 1'] = helper['TEAM 1'].str.replace('West Green', 'Wests Green')
helper['TEAM 2'] = helper['TEAM 2'].str.replace('West Green', 'Wests Green')


In [4]:
helper['TEAM 2'].unique()

array(['Wests Green', 'University', 'Wests', 'Wests Red', 'Souths', 'nan',
       'University Seapigs', 'Maitland', 'Port Stephens', 'Tigers Yellow',
       'University 1', 'Tigers', 'University Gentlemen', 'Norths', 'BYE',
       'TEAM 2', 'Gosford', 'University 2', 'Tigers 1', 'Crusaders',
       'Tigers Black', 'EASTER', 'Colts'], dtype=object)

In [10]:
import pandas as pd

# Remove unwanted team labels
cleaned = helper[
    ~helper['TEAM 1'].isin(['EASTER', 'TEAM 1', 'TEAM 2']) &
    ~helper['TEAM 2'].isin(['EASTER', 'TEAM 1', 'TEAM 2'])
].copy()

# Create a standardised pairing regardless of order
cleaned['Team_A'] = cleaned[['TEAM 1', 'TEAM 2']].min(axis=1)
cleaned['Team_B'] = cleaned[['TEAM 1', 'TEAM 2']].max(axis=1)

# Now group by GRADE and the unordered pair, and count games
match_counts = (
    cleaned
    .groupby(['GRADE', 'Team_A', 'Team_B'])
    .size()
    .reset_index(name='Matches')
)

# Now for each GRADE, pivot into a symmetric table
grades = match_counts['GRADE'].unique()
for grade in grades:
    if grade == '5th':
        sub = match_counts[match_counts['GRADE'] == grade]

        # Get full list of teams involved in this grade
        teams = pd.unique(sub[['Team_A', 'Team_B']].values.ravel())

        # Create full symmetric index and columns
        pivot = sub.pivot(index='Team_A', columns='Team_B', values='Matches').fillna(0).astype(int)
        pivot = pivot.reindex(index=teams, columns=teams, fill_value=0)

        # Symmetrize the matrix by copying upper + lower triangle
        symmetrical = pivot.add(pivot.T, fill_value=0).astype(int)

        # Compute total games played by team
        symmetrical['Total'] = symmetrical.sum(axis=1)
        symmetrical.loc['Total'] = symmetrical.sum(axis=0)

        print(f"\nGRADE: {grade}")
        print(symmetrical)



GRADE: 5th
Team_B              BYE  Colts  Crusaders  Maitland  Norths  Port Stephens  \
Team_A                                                                       
BYE                   0      4          3         3       3              3   
Colts                 4      0          2         2       2              2   
Crusaders             3      2          0         2       2              2   
Maitland              3      2          2         0       2              2   
Norths                3      2          2         2       0              2   
Port Stephens         3      2          2         2       2              0   
Tigers Black          3      2          2         2       2              2   
Tigers Yellow         3      2          1         2       2              2   
University            2      2          2         2       2              1   
University Seapigs    4      2          2         2       2              1   
Wests                 1      2          2         2 

In [6]:
# Function to check home games for a team
def check_home_games(team, home_ground):
    team_games = cleaned[
        ((cleaned['TEAM 1'] == team) & (cleaned['FIELD'] == home_ground)) |
        ((cleaned['TEAM 2'] == team) & (cleaned['FIELD'] == home_ground))
    ]
    
    # Group by grade and opponent
    home_game_counts = team_games.groupby(['GRADE', 'Team_A', 'Team_B']).size().reset_index(name='Home_Games')
    
    # Filter for games involving the team
    home_game_counts = home_game_counts[
        (home_game_counts['Team_A'] == team) | (home_game_counts['Team_B'] == team)
    ]
    
    # Check if home games are exactly half of total games
    for _, row in home_game_counts.iterrows():
        grade = row['GRADE']
        opponent = row['Team_B'] if row['Team_A'] == team else row['Team_A']
        total_games = match_counts[
            (match_counts['GRADE'] == grade) &
            (((match_counts['Team_A'] == team) & (match_counts['Team_B'] == opponent)) |
             ((match_counts['Team_B'] == team) & (match_counts['Team_A'] == opponent)))
        ]['Matches'].values[0]
        
        if row['Home_Games'] != total_games / 2:
            print(f"Discrepancy found: {team} vs {opponent} in grade {grade} - Home games: {row['Home_Games']}, Total games: {total_games}")

# Check for Maitland
check_home_games('Maitland', 'MAI')

# Check for Gosford
check_home_games('Gosford', 'CC')

Discrepancy found: Maitland vs Colts in grade 4th - Home games: 2, Total games: 3
Discrepancy found: Maitland vs Norths in grade 5th - Home games: 2, Total games: 2
Discrepancy found: Maitland vs Tigers Yellow in grade 5th - Home games: 2, Total games: 2
Discrepancy found: Maitland vs Colts in grade 6th - Home games: 2, Total games: 3
Discrepancy found: Maitland vs Crusaders in grade 6th - Home games: 1, Total games: 3
Discrepancy found: Maitland vs Souths in grade 6th - Home games: 1, Total games: 3
Discrepancy found: Maitland vs Tigers in grade 6th - Home games: 2, Total games: 3
Discrepancy found: Maitland vs University Gentlemen in grade 6th - Home games: 1, Total games: 3
Discrepancy found: Maitland vs Wests in grade 6th - Home games: 2, Total games: 3
Discrepancy found: Maitland vs Souths in grade PHL - Home games: 1, Total games: 4
Discrepancy found: Gosford vs Tigers in grade PHL - Home games: 3, Total games: 4


In [14]:
first_team = 'Souths'
second_team = 'Maitland'
grade = 'PHL'
helper[(((helper['TEAM 1'] == first_team) & (helper['TEAM 2'] == second_team) ) | ((helper['TEAM 2'] == first_team) & (helper['TEAM 1'] == second_team) )) & (helper['GRADE'] ==  grade)]
# helper[(((helper['TEAM 1'] == first_team)  ) | ((helper['TEAM 2'] == first_team) )) & (helper['GRADE'] ==  grade)]


,DAY,DATE,TIME,FIELD,ROUND,GRADE,TEAM 1,v,TEAM 2
14,SUN,23-March-2025,16:00,WF,1,PHL,Souths,v,Maitland
314,FRI,30-May-2025,19:00,EF,9,PHL,Souths,v,Maitland
341,SUN,01-June-2025,15:00,MAI,5,PHL,Maitland,v,Souths
702,SUN,10-August-2025,16:00,WF,18,PHL,Souths,v,Maitland


In [12]:
first_team = 'Souths'
second_team = 'Maitland'
grade = '2nd'
helper[(((helper['TEAM 1'] == first_team) & (helper['TEAM 2'] == second_team) ) | ((helper['TEAM 2'] == first_team) & (helper['TEAM 1'] == second_team) )) & (helper['GRADE'] ==  grade)]
# helper[(((helper['TEAM 1'] == first_team)  ) | ((helper['TEAM 2'] == first_team) )) & (helper['GRADE'] ==  grade)]


,DAY,DATE,TIME,FIELD,ROUND,GRADE,TEAM 1,v,TEAM 2
6,SUN,23-March-2025,17:30,EF,1,2nd,Maitland,v,Souths
340,SUN,01-June-2025,13:30,MAI,9,2nd,Maitland,v,Souths
614,SAT,26-July-2025,17:00,MAI,5,2nd,Maitland,v,Souths
701,SUN,10-August-2025,14:30,WF,18,2nd,Souths,v,Maitland
